## Prediction de salaire

### Bibliothèques utiles

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

pd.set_option("display.float_format", lambda x: f"{x:,.3f}")
sns.set_context("talk")
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 200)


### I-  Description de la base de donnée

#### Salaires dans le secteur privé selon le sexe et la catégorie socioprofessionnelle (base communale)

* Le champ correspond aux salariés du privé, y compris bénéficiaires de contrats aidés et de contrats de professionnalisation ; hors apprentis, stagiaires, salariés agricoles et salariés des particuliers employeurs.

* Les données sur les salaires au lieu de travail sont ventilées selon le sexe et la catégorie socioprofessionnelle (hors agriculture), et détaillées par territoire : commune, arrondissement municipal, arrondissement, aire d'attraction des villes 2020, bassin de vie 2022, établissement public de coopération intercommunal, unité urbaine 2020, zone d'emploi 2020, département, région, France hors Mayotte.

Variables explicatives retenues :

SEX : Sexe (Homme/Femme) - impact attendu sur l'écart salarial

PCS_ESE : Profession et Catégorie Socioprofessionnelle - déterminant principal

TIME_PERIOD : Année (2022-2023) - évolution temporelle

GEO : Code géographique - variations territoriales

In [20]:
## lien vers le dataset
data_path = r"dataset\DS_BTS_SAL_EQTP_SEX_PCS_2023_data.csv"
metadata_path = r"dataset\DS_BTS_SAL_EQTP_SEX_PCS_2023_metadata.csv"


In [21]:
data = pd.read_csv(data_path, sep=';')
metadata = pd.read_csv(metadata_path, sep=';')

In [22]:
# Affichage des informations de base
print(f" Dimensions du dataset: {data.shape}")
print(f" Colonnes disponibles: {list(data.columns)}")

 Dimensions du dataset: (370710, 9)
 Colonnes disponibles: ['GEO', 'GEO_OBJECT', 'FREQ', 'SEX', 'PCS_ESE', 'DERA_MEASURE', 'CONF_STATUS', 'TIME_PERIOD', 'OBS_VALUE']


In [23]:
print("\n Aperçu des premières lignes:")
print(data.head())


 Aperçu des premières lignes:
     GEO GEO_OBJECT FREQ SEX PCS_ESE                      DERA_MEASURE CONF_STATUS  TIME_PERIOD  OBS_VALUE
0  26362     BV2022    A   F      _T  SALAIRE_NET_EQTP_MENSUEL_MOYENNE           F         2022  2,157.285
1  26324     BV2022    A  _T       4  SALAIRE_NET_EQTP_MENSUEL_MOYENNE           F         2022  3,112.938
2  26307     BV2022    A  _T       6  SALAIRE_NET_EQTP_MENSUEL_MOYENNE           F         2023  2,013.097
3  26362     BV2022    A  _T      _T  SALAIRE_NET_EQTP_MENSUEL_MOYENNE           F         2023  2,483.037
4  27170     BV2022    A   F       4  SALAIRE_NET_EQTP_MENSUEL_MOYENNE           F         2022  2,107.221


In [24]:
print(" Types de données:")
print(data.dtypes)

 Types de données:
GEO              object
GEO_OBJECT       object
FREQ             object
SEX              object
PCS_ESE          object
DERA_MEASURE     object
CONF_STATUS      object
TIME_PERIOD       int64
OBS_VALUE       float64
dtype: object


In [25]:

print("\n📋 Informations détaillées:")
print(data.info())


📋 Informations détaillées:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370710 entries, 0 to 370709
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   GEO           370710 non-null  object 
 1   GEO_OBJECT    370710 non-null  object 
 2   FREQ          370710 non-null  object 
 3   SEX           370710 non-null  object 
 4   PCS_ESE       370710 non-null  object 
 5   DERA_MEASURE  370710 non-null  object 
 6   CONF_STATUS   370710 non-null  object 
 7   TIME_PERIOD   370710 non-null  int64  
 8   OBS_VALUE     350250 non-null  float64
dtypes: float64(1), int64(1), object(7)
memory usage: 25.5+ MB
None


In [26]:
# Analyse des variables uniques
print("📊 Analyse des variables catégorielles:")
categorical_vars = ['GEO_OBJECT', 'FREQ', 'SEX', 'PCS_ESE', 'DERA_MEASURE', 'CONF_STATUS']

for var in categorical_vars:
    unique_vals = data[var].unique()
    print(f"\n{var} ({len(unique_vals)} valeurs uniques):")
    print(f"  Valeurs: {unique_vals[:10]}...")  # Afficher les 10 premières valeurs

print(f"\n📈 Variable cible (OBS_VALUE) - Statistiques descriptives:")
print(data['OBS_VALUE'].describe())

print(f"\n📅 Période temporelle:")
print(f"  Années disponibles: {sorted(data['TIME_PERIOD'].unique())}")

print(f"\n❌ Valeurs manquantes:")
missing_data = data.isnull().sum()
print(missing_data[missing_data > 0])


📊 Analyse des variables catégorielles:

GEO_OBJECT (11 valeurs uniques):
  Valeurs: ['BV2022' 'ARR' 'AAV2020' 'COM' 'UU2020' 'ARM' 'EPCI' 'ZE2020' 'DEP'
 'FRANCE']...

FREQ (1 valeurs uniques):
  Valeurs: ['A']...

SEX (3 valeurs uniques):
  Valeurs: ['F' '_T' 'M']...

PCS_ESE (5 valeurs uniques):
  Valeurs: ['_T' '4' '6' '5' '1T3']...

DERA_MEASURE (1 valeurs uniques):
  Valeurs: ['SALAIRE_NET_EQTP_MENSUEL_MOYENNE']...

CONF_STATUS (2 valeurs uniques):
  Valeurs: ['F' 'C']...

📈 Variable cible (OBS_VALUE) - Statistiques descriptives:
count   350,250.000
mean      2,445.223
std         758.391
min         793.399
25%       1,908.283
50%       2,191.887
75%       2,671.732
max      14,047.315
Name: OBS_VALUE, dtype: float64

📅 Période temporelle:
  Années disponibles: [np.int64(2022), np.int64(2023)]

❌ Valeurs manquantes:
OBS_VALUE    20460
dtype: int64


### II-  Analyse exploratoire de donnée

#### A analyse univarié

In [14]:
# Nettoyage initial - suppression des lignes avec valeurs manquantes pour OBS_VALUE
df_clean = data.dropna(subset=['OBS_VALUE']).copy()
print(f"📋 Après suppression des valeurs manquantes: {df_clean.shape[0]} lignes")

# Décodage des variables catégorielles pour mieux les comprendre
print("\n📊 ANALYSE UNIVARIÉE:")
print("-" * 30)

# Variable cible
print("🎯 Variable cible (OBS_VALUE - Salaire):")
print(f"  Moyenne: {df_clean['OBS_VALUE'].mean():.2f}€")
print(f"  Médiane: {df_clean['OBS_VALUE'].median():.2f}€")
print(f"  Écart-type: {df_clean['OBS_VALUE'].std():.2f}€")
print(f"  Min: {df_clean['OBS_VALUE'].min():.2f}€")
print(f"  Max: {df_clean['OBS_VALUE'].max():.2f}€")


📋 Après suppression des valeurs manquantes: 350250 lignes

📊 ANALYSE UNIVARIÉE:
------------------------------
🎯 Variable cible (OBS_VALUE - Salaire):
  Moyenne: 2445.22€
  Médiane: 2191.89€
  Écart-type: 758.39€
  Min: 793.40€
  Max: 14047.32€


In [15]:

# Distribution par sexe
print("\n👥 Distribution par SEXE:")
sex_stats = df_clean.groupby('SEX')['OBS_VALUE'].agg(['count', 'mean', 'median', 'std'])
print(sex_stats)

# Mapping pour décoder les valeurs
sex_mapping = {'F': 'Femme', 'M': 'Homme', '_T': 'Total/Ensemble'}
pcs_mapping = {'1T3': 'Cadres', '4': 'Prof_intermédiaires', '5': 'Employés', 
               '6': 'Ouvriers', '_T': 'Ensemble'}

df_clean['SEX_decoded'] = df_clean['SEX'].map(sex_mapping)
df_clean['PCS_ESE_decoded'] = df_clean['PCS_ESE'].map(pcs_mapping)

print("\n💼 Distribution par CATÉGORIE SOCIOPROFESSIONNELLE (PCS_ESE):")
pcs_stats = df_clean.groupby('PCS_ESE_decoded')['OBS_VALUE'].agg(['count', 'mean', 'median', 'std'])
print(pcs_stats.sort_values('mean', ascending=False))


👥 Distribution par SEXE:
      count      mean    median     std
SEX                                    
F    116750 2,277.065 2,054.944 650.718
M    116750 2,586.463 2,325.121 825.195
_T   116750 2,472.141 2,217.163 756.405

💼 Distribution par CATÉGORIE SOCIOPROFESSIONNELLE (PCS_ESE):
                     count      mean    median     std
PCS_ESE_decoded                                       
Cadres               70050 3,713.431 3,668.245 603.721
Prof_intermédiaires  70050 2,474.066 2,441.476 285.048
Ensemble             70050 2,258.867 2,200.938 322.249
Ouvriers             70050 1,911.016 1,915.633 209.056
Employés             70050 1,868.733 1,848.270 144.475


#### B Analyse bivariée

In [16]:
# Analyse de l'écart salarial par sexe et CSP
print("💰 Écart salarial Homme/Femme par catégorie:")
pivot_sex_pcs = df_clean[df_clean['SEX'].isin(['F', 'M'])].pivot_table(
    values='OBS_VALUE', 
    index='PCS_ESE_decoded', 
    columns='SEX', 
    aggfunc='mean'
)
pivot_sex_pcs['Écart_H_F'] = pivot_sex_pcs['M'] - pivot_sex_pcs['F']
pivot_sex_pcs['Écart_%'] = (pivot_sex_pcs['Écart_H_F'] / pivot_sex_pcs['F']) * 100
print(pivot_sex_pcs.round(2))


💰 Écart salarial Homme/Femme par catégorie:
SEX                         F         M  Écart_H_F  Écart_%
PCS_ESE_decoded                                            
Cadres              3,399.610 3,966.410    566.790   16.670
Employés            1,830.140 1,922.680     92.550    5.060
Ensemble            2,097.630 2,400.770    303.140   14.450
Ouvriers            1,751.640 2,012.570    260.930   14.900
Prof_intermédiaires 2,306.310 2,629.890    323.580   14.030


In [18]:
# Évolution temporelle
print("\n📅 Évolution des salaires par année:")
temporal_analysis = df_clean.groupby(['TIME_PERIOD', 'SEX_decoded'])['OBS_VALUE'].mean().unstack()
print(temporal_analysis.round(2))

# Détection des outliers avec la méthode IQR
print("\n🔍 DÉTECTION DES OUTLIERS:")
print("-" * 30)
Q1 = df_clean['OBS_VALUE'].quantile(0.25)
Q3 = df_clean['OBS_VALUE'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df_clean[(df_clean['OBS_VALUE'] < lower_bound) | 
                   (df_clean['OBS_VALUE'] > upper_bound)]

print(f"📊 Nombre d'outliers détectés: {len(outliers)} ({len(outliers)/len(df_clean)*100:.1f}%)")
print(f"📊 Seuil inférieur: {lower_bound:.2f}€")
print(f"📊 Seuil supérieur: {upper_bound:.2f}€")

if len(outliers) > 0:
    print(f"📊 Outliers les plus extrêmes:")
    print(outliers.nlargest(5, 'OBS_VALUE')[['SEX_decoded', 'PCS_ESE_decoded', 'OBS_VALUE', 'TIME_PERIOD']])


📅 Évolution des salaires par année:
SEX_decoded     Femme     Homme  Total/Ensemble
TIME_PERIOD                                    
2022        2,237.830 2,549.740       2,434.670
2023        2,316.300 2,623.180       2,509.610

🔍 DÉTECTION DES OUTLIERS:
------------------------------
📊 Nombre d'outliers détectés: 28517 (8.1%)
📊 Seuil inférieur: 763.11€
📊 Seuil supérieur: 3816.91€
📊 Outliers les plus extrêmes:
           SEX_decoded      PCS_ESE_decoded  OBS_VALUE  TIME_PERIOD
17317            Homme  Prof_intermédiaires 14,047.315         2023
15260   Total/Ensemble  Prof_intermédiaires 10,864.653         2023
41951            Homme  Prof_intermédiaires  9,648.090         2022
319056           Homme               Cadres  9,618.147         2023
189255           Homme               Cadres  9,241.599         2022


### III- Modèle de prédiction 

#### A nettoyage de donnée

In [27]:
# Filtrage pour ne garder que les données pertinentes pour la modélisation
print("🔄 Nettoyage et préparation:")

# Exclusion des lignes "Total/Ensemble" pour éviter la redondance
df_model = df_clean[
    (df_clean['SEX'] != '_T') & 
    (df_clean['PCS_ESE'] != '_T')
].copy()

print(f"📋 Dataset après exclusion des totaux: {df_model.shape[0]} lignes")

# Traitement des outliers - Application d'une transformation log pour réduire l'impact
print("📊 Traitement des outliers:")
print(f"  Avant: Min={df_model['OBS_VALUE'].min():.2f}, Max={df_model['OBS_VALUE'].max():.2f}")

# Winsorisation pour limiter les valeurs extrêmes
from scipy.stats import mstats
df_model['OBS_VALUE_winsorized'] = mstats.winsorize(df_model['OBS_VALUE'], limits=[0.01, 0.01])
print(f"  Après winsorisation: Min={df_model['OBS_VALUE_winsorized'].min():.2f}, Max={df_model['OBS_VALUE_winsorized'].max():.2f}")

# Encodage des variables catégorielles
print("\n🔤 Encodage des variables catégorielles:")

# Label Encoding pour les variables ordinales
le_sex = LabelEncoder()
le_pcs = LabelEncoder()

df_model['SEX_encoded'] = le_sex.fit_transform(df_model['SEX'])
df_model['PCS_ESE_encoded'] = le_pcs.fit_transform(df_model['PCS_ESE'])

print(f"  SEX mapping: {dict(zip(le_sex.classes_, le_sex.transform(le_sex.classes_)))}")
print(f"  PCS_ESE mapping: {dict(zip(le_pcs.classes_, le_pcs.transform(le_pcs.classes_)))}")

# Création de variables dummy pour l'interprétation
df_model_dummies = pd.get_dummies(df_model, columns=['SEX', 'PCS_ESE'], prefix=['SEX', 'PCS'])
print(f"  Variables après création des dummies: {df_model_dummies.shape[1]} colonnes")

# Sélection des features pour le modèle
features_encoded = ['SEX_encoded', 'PCS_ESE_encoded', 'TIME_PERIOD', 'GEO']
features_dummies = [col for col in df_model_dummies.columns if col.startswith(('SEX_', 'PCS_'))]
features_dummies.append('TIME_PERIOD')

print(f"  Features sélectionnées (encoded): {features_encoded}")
print(f"  Nombre de features (dummies): {len(features_dummies)}")

# Préparation des données pour GEO (simplification)
# Conversion GEO en numérique pour la modélisation
df_model['GEO_numeric'] = pd.to_numeric(df_model['GEO'], errors='coerce')
df_model['GEO_numeric'].fillna(df_model['GEO_numeric'].median(), inplace=True)

features_final = ['SEX_encoded', 'PCS_ESE_encoded', 'TIME_PERIOD', 'GEO_numeric']

print(f"\n✅ Dataset final prêt pour la modélisation:")
print(f"  Nombre d'observations: {df_model.shape[0]}")
print(f"  Features utilisées: {features_final}")
print(f"  Variable cible: OBS_VALUE_winsorized")

🔄 Nettoyage et préparation:
📋 Dataset après exclusion des totaux: 186800 lignes
📊 Traitement des outliers:
  Avant: Min=793.40, Max=14047.32
  Après winsorisation: Min=1536.42, Max=4849.43

🔤 Encodage des variables catégorielles:
  SEX mapping: {'F': np.int64(0), 'M': np.int64(1)}
  PCS_ESE mapping: {'1T3': np.int64(0), '4': np.int64(1), '5': np.int64(2), '6': np.int64(3)}
  Variables après création des dummies: 18 colonnes
  Features sélectionnées (encoded): ['SEX_encoded', 'PCS_ESE_encoded', 'TIME_PERIOD', 'GEO']
  Nombre de features (dummies): 11

✅ Dataset final prêt pour la modélisation:
  Nombre d'observations: 186800
  Features utilisées: ['SEX_encoded', 'PCS_ESE_encoded', 'TIME_PERIOD', 'GEO_numeric']
  Variable cible: OBS_VALUE_winsorized


C:\Users\User\AppData\Local\Temp\ipykernel_28072\100626605.py:49: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_model['GEO_numeric'].fillna(df_model['GEO_numeric'].median(), inplace=True)


#### Modélisation de donnée

In [28]:


# Préparation des données pour l'entraînement
X = df_model[features_final]
y = df_model['OBS_VALUE_winsorized']

print("📊 Vérification des données d'entrée:")
print(f"  Shape de X: {X.shape}")
print(f"  Shape de y: {y.shape}")
print(f"  Valeurs manquantes dans X: {X.isnull().sum().sum()}")
print(f"  Valeurs manquantes dans y: {y.isnull().sum()}")



📊 Vérification des données d'entrée:
  Shape de X: (186800, 4)
  Shape de y: (186800,)
  Valeurs manquantes dans X: 0
  Valeurs manquantes dans y: 0


In [29]:
# Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=df_model['PCS_ESE_encoded']
)

print(f"\n📈 Split des données:")
print(f"  Taille train: {X_train.shape[0]} ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"  Taille test: {X_test.shape[0]} ({X_test.shape[0]/len(X)*100:.1f}%)")


📈 Split des données:
  Taille train: 149440 (80.0%)
  Taille test: 37360 (20.0%)


In [30]:

# Standardisation des features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\n🔧 Standardisation effectuée")
print(f"  Moyennes après standardisation: {X_train_scaled.mean(axis=0).round(3)}")
print(f"  Écarts-types après standardisation: {X_train_scaled.std(axis=0).round(3)}")




🔧 Standardisation effectuée
  Moyennes après standardisation: [-0.  0.  0. -0.]
  Écarts-types après standardisation: [1. 1. 1. 1.]


In [31]:

# MODÈLE 1: Régression Linéaire
print(f"\n🎯 MODÈLE 1: RÉGRESSION LINÉAIRE")
print("-" * 40)
print("💭 Justification du choix:")
print("   - Variable cible continue (salaire)")
print("   - Relation potentiellement linéaire entre features et salaire")
print("   - Interprétabilité élevée des coefficients")
print("   - Baseline simple et robuste")

# Entraînement du modèle de régression linéaire
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

# Prédictions
y_pred_train_lr = lr_model.predict(X_train_scaled)
y_pred_test_lr = lr_model.predict(X_test_scaled)

print(f"\n📊 Coefficients du modèle:")
for i, feature in enumerate(features_final):
    print(f"  {feature}: {lr_model.coef_[i]:.3f}")
print(f"  Intercept: {lr_model.intercept_:.3f}")

# Interprétation des coefficients
print(f"\n🔍 Interprétation des coefficients:")
coef_interpretation = {
    'SEX_encoded': f"Être homme (vs femme) : {lr_model.coef_[0]:.0f}€ de différence",
    'PCS_ESE_encoded': f"Changement de catégorie PCS : {lr_model.coef_[1]:.0f}€ par niveau",
    'TIME_PERIOD': f"Évolution annuelle : {lr_model.coef_[2]:.0f}€ par an",
    'GEO_numeric': f"Impact géographique : {lr_model.coef_[3]:.6f}€ par unité geo"
}

for feature, interpretation in coef_interpretation.items():
    print(f"  • {interpretation}")


🎯 MODÈLE 1: RÉGRESSION LINÉAIRE
----------------------------------------
💭 Justification du choix:
   - Variable cible continue (salaire)
   - Relation potentiellement linéaire entre features et salaire
   - Interprétabilité élevée des coefficients
   - Baseline simple et robuste

📊 Coefficients du modèle:
  SEX_encoded: 150.154
  PCS_ESE_encoded: -662.490
  TIME_PERIOD: 35.157
  GEO_numeric: -2.323
  Intercept: 2473.773

🔍 Interprétation des coefficients:
  • Être homme (vs femme) : 150€ de différence
  • Changement de catégorie PCS : -662€ par niveau
  • Évolution annuelle : 35€ par an
  • Impact géographique : -2.323123€ par unité geo
